# Pre Processing Data for ML Training

## Introduction

This notebook is dependent on the execution of `scripts/02 - exploratory_data_analysis.py`

## Imports and Definitions

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib
import yaml
import os

print("Defining Classes")

class Logger:
    def __init__(self, config):
        self.config = config
        self.log_dir = config['logging']['out_dir']
        self.tag = config['base']['tag']
        self.file_path = os.path.join('outputs', self.tag, self.log_dir, 'log.txt')
        self.verbose = config['logging']['verbose']
        
    def log(self, message):
        current_datetime = datetime.datetime.now()
        datetime_string = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        log_message = f"{datetime_string}: {message}"
        if self.verbose:
            print(log_message)
        with open(self.file_path, "a") as f:
            f.write(f'{log_message}\n')

print("Defining Functions")

def loan_status_to_int(status):
    if status == 'Charged Off':
        return 0
    if status == 'Fully Paid':
        return 1
    else:
        return -1

def bestbandwidth(data):
    return 1.06*np.std(data)*len(data)**(-1/5)

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return f"Created directory: {directory}"
    else:
        return f"Directory already exists: {directory}"

def transform_type(sqlite_type):
    if sqlite_type == 'INTEGER':
        return 'int'
    if sqlite_type == 'REAL':
        return 'float'
    if sqlite_type == 'TEXT':
        return 'object'

def map_dtype_to_sqlite(col_type):
    if col_type.startswith('int') or col_type == 'bool':
        return 'INTEGER'
    elif col_type.startswith('float'):
        return 'REAL'
    else:  # Default case, particularly for 'object' and other unhandled types
        return 'TEXT'

print("Reading Config File")

config_file_path = '../config/config.yml'

root_path = '..'

print(f"Reading Config File {config_file_path}")
with open(config_file_path, 'r') as f:
    config = yaml.safe_load(f)

print("Defining Variables and Creating Directories")

sqlite_file = config['data']['output_sqlite']

tag = config['base']['tag']

git_repo = config['base']['git_repo']

fontsize = config['plotting']['fontsize']
figsize_x = config['plotting']['figure_xsize']
figsize_y = config['plotting']['figure_ysize']

out_dir_figures = f"outputs/{tag}/figures"

out_dir_stats = f"outputs/{tag}/stats"

out_dir_log = f"outputs/{tag}/log"

sqlite_file = os.path.join(f'{root_path}/outputs/{tag}/data/{sqlite_file}')
out_dir_figures = os.path.join(root_path, out_dir_figures)
out_dir_stats = os.path.join(root_path, out_dir_stats)
out_dir_log = os.path.join(root_path, out_dir_log)

columns_of_interest = config['base']['columns_of_interest']

print("Done with initial setup")

## Loading and Cleaning Data and Defining Columns

In [ ]:
print("Loading Data")
# Defining the connection to the database
conn = sqlite3.connect(sqlite_file)

# Loading descriptions into dataframe
description_fetch_query = f"""SELECT *
                    FROM descriptions
                    """
descriptions = pd.read_sql_query(description_fetch_query, conn, index_col = 'name')

column_types = {idx:transform_type(row['data_type']) for idx, row in descriptions.iterrows() if row['location'] == 'loans_data' and idx in columns_of_interest}

# Loading data into dataframe
data_fetch_query = f"""SELECT {', '.join(columns_of_interest)} 
                       FROM loans_data
                       ORDER BY RANDOM()"""

loans_data = pd.read_sql_query(data_fetch_query, conn, index_col='id', dtype=column_types)

# Closing connection
conn.close()

print("Filtering known bad columns")

loans_data = loans_data[loans_data['issue_d_unix'] != 0]

print("Creating columns")

loans_data['issue_d'] = pd.to_datetime(loans_data['issue_d_unix'], unit='s')
descriptions = pd.concat([descriptions, pd.DataFrame({
    'name':['issue_d'],
    'full_name': ['Issue Date'],
    'type': ['Column'],
    'location': ['loans_data'],
    'description': ['Date the loan was issued'],
    'data_type': ['TEXT']
}).set_index('name')])

loans_data['issue_month'] = loans_data['issue_d'].apply(lambda x: x.month)
descriptions = pd.concat([descriptions, pd.DataFrame({
    'name':['issue_month'],
    'full_name': ['Issue Month'],
    'type': ['Column'],
    'location': ['loans_data'],
    'description': ['Month of the year the loan was issued'],
    'data_type': ['INT']
}).set_index('name')])

print("Limiting dataset to two types of Loan Status only")

loans_data = loans_data[(loans_data['loan_status'] == 'Charged Off') | (loans_data['loan_status'] == 'Fully Paid')]

loans_data['loan_status'] = loans_data['loan_status'].apply(lambda x: x == 'Charged Off')

print("Done with Loading and Cleaning Data and Defining Columns")

## Pre-Processing Data

### Preparing Numerical and Dummy Columns, and Dropping Unecessary Ones

In [ ]:
print("Declaring Numerical Columns")
numerical_columns = ['loan_status', 'loan_amnt', 'int_rate', 'installment', 'annual_inc', 
                     'dti', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
                     'mort_acc', 'pub_rec_bankruptcies']

print("Declaring Dummy Columns")
dummy_columns = ['term_months', 'sub_grade', 'home_ownership', 'verification_status',
                 'purpose', 'initial_list_status', 'application_type']

print("Declaring Columns to Drop")
drop_columns = ['grade', 'issue_d', 'issue_d_unix']

print("Dropping Non-Interesting Columns")
loans_data_ML = loans_data.drop(drop_columns, axis='columns')
print("Creating Dummy Columns")
loans_data_ML = pd.get_dummies(loans_data_ML, columns=dummy_columns, drop_first=True)

print("Done Preparing Numerical and Dummy Columns, and Dropping Unecessary Ones")
loans_data_ML

### Saving ML Prepared Data to new table

In [ ]:
print(f"Connecting to Database at {sqlite_file}")
conn = sqlite3.connect(sqlite_file)

print("Creating Queries")
drop_loans_data_ML_query = 'DROP TABLE IF EXISTS loans_data_ML'

# Query to create a table for the loans data
create_loans_data_ML_table_query = 'CREATE TABLE loans_data_ML (' + ', '.join([f"\"{col}\" {col_type}" for col, col_type in zip(loans_data_ML.columns, [map_dtype_to_sqlite(str(loans_data_ML[col].dtype)) for col in loans_data_ML.columns])]) + ')'

print("Dropping old tables and creating new ones")
# Drops and creates the tables
conn.execute(drop_loans_data_ML_query)
conn.execute(create_loans_data_ML_table_query)

print("Loading data into tables")
# Insert data from DataFrame to the SQLite table
loans_data_ML.to_sql('loans_data_ML', conn, if_exists='replace', index=True)

conn.close()
print("Done With Constructing the Database File")